In [ ]:
import calendar
import time
import datetime
import pandas as pd
import numpy as np
from itertools import compress
import stanager.scheduling as ss
# import sys
# sys.path.insert(0, './stanager')
# import scheduling

# Scheduling排课模块

## 定义课节

In [ ]:
JYZJ = ss.Classes('九阴真经', duration=40)

In [ ]:
# mentor_names = ['CHR', 'HL', 'FJ', 'YB', 'ZBF', 'JSJ', 'LH', 'WXY', 'QHD']
# mentors = {}
# for mt in mentor_names:
#     mentors[mt] = pd.read_excel('../../Python_1on1/Mentors_available_time.xlsx', sheet_name=mt, index_col=0)\
#     .drop(['周', '每日空闲时间', '最少可排课时'], axis = 0).drop(['Unnamed: 14'], axis = 1)
# mentors['HL']

In [ ]:
# date = mentors['HL'].columns
# time = mentors['HL'].index
# time

# # for x in time:
# #     for y in date:
# #         if mentors['HL'].loc[x, y] == 1:
            
# # avlb = 
# # avlb == 1

In [ ]:
# start_h, end_h = tuple(datetime.datetime.strptime(x, '%H:%M').hour for x in mentors['HL'].index[0].split('-'))
# start = [x + datetime.timedelta(hours = start_h) for x in mentors['HL'].columns]

## 定义讲师邱处机

In [ ]:
start_1 = '2019-02-03 19:00'
end_1 = '2019-02-03 21:00'
start_2 = '2019-02-05 19:00'
end_2 = '2019-02-05 21:00'
qiu_chuji = ss.Mentor(name = '邱处机', available = [(start_1, end_1),
                                             (start_2, end_2)])
print('讲师姓名：{}'.format(qiu_chuji.name))
print('讲师可排课时间：{}'.format(qiu_chuji.available))

## 定义学生

定义学生杨康

In [ ]:
yang_kang = ss.Student('杨康', [('2019-02-03 14:00', '2019-02-03 18:00'),
                         ('2019-02-03 20:00', '2019-02-03 22:00'),
                         ('2019-02-05 20:00', '2019-02-05 22:00')])

定义学生杨过

In [ ]:
yang_guo = ss.Student('杨过', [('2019-02-03 20:00', '2019-02-03 22:00'),
                         ('2019-02-05 14:00', '2019-02-05 16:00'),
                         ('2019-02-05 20:00', '2019-02-05 22:00')])

## 预约课程

初次预约课程

In [ ]:
qiu_chuji.init_appointment(JYZJ, yang_kang, start= '2019-02-03 19:30')
print('{}老师，你的课程表如下：\n{}\n请提前做好准备！'.format(qiu_chuji.name, qiu_chuji.schedule))
print('老师的剩余可用时间为\n{}'.format(qiu_chuji.available))

给第二个学生预约课程：

In [ ]:
qiu_chuji.init_appointment(JYZJ, yang_guo, start= '2019-02-03 20:20')
print('{}老师，你的课程表如下：\n{}\n请提前做好准备！'.format(qiu_chuji.name, qiu_chuji.schedule))
print('老师的剩余可用时间为\n{}'.format(qiu_chuji.available))

杨康同学要排第二节课

In [ ]:
qiu_chuji.init_appointment(JYZJ, yang_kang, start= '2019-02-05 20:20')
print('{}老师，你的课程表如下：\n{}\n请提前做好准备！'.format(qiu_chuji.name, qiu_chuji.schedule))
print('老师的剩余可用时间为\n{}'.format(qiu_chuji.available))

杨康同学要调课

In [ ]:
qiu_chuji.change_appointment(yang_kang, 0, '2019-02-05 19:00')
print('{}老师，你的课程表如下：\n{}\n请提前做好准备！'.format(qiu_chuji.name, qiu_chuji.schedule))
print('老师的剩余可用时间为\n{}'.format(qiu_chuji.available))

杨康同学的课程表：

In [ ]:
yang_kang.schedule = qiu_chuji.schedule[yang_kang.name]
yang_kang.schedule

杨过同学的课表：

In [ ]:
yang_guo.schedule = qiu_chuji.schedule[yang_guo.name]
yang_guo.schedule

## TODO

* 如何匹配合适的老师？

* 如何找到一段合适的上课时间？

## TODO：排课表release版

In [ ]:
start = '2019-03-27'
end = '2019-06-26'

In [ ]:
daterange = ss.date_range(start, end)
parsed_start = ss.parse_time(start, str_format = '%Y-%m-%d')
calendar.weekday(parsed_start.year, parsed_start.month, parsed_start.day)
sch_day = []
target = calendar.weekheader(3).split(' ').index('Tue')
for i in range(6, len(daterange),7):
    sch_day.append(daterange[i])
sch_day[:12]

# communicate模块，群发个性化学习任务

In [1]:
import pandas as pd
import itchat as ic
import stanager.communicate as sc

## 登录微信

In [2]:
ic.auto_login()

Getting uuid of QR code.
Please scan the QR code to log in.
Please press confirm on your phone.
Loading the contact, this may take a little while.
Login successfully as Udacity-Tyelcie


## 读取任务列表

根据每个人当前的进度，制定了不同的任务，然后私信发给各位同学。用`pd.read_csv`或`pd.read_excel`也行，但一般都存在设密的石墨文档上，转存也有点麻烦。所以干脆把要抓的内容选中，Ctrl+C，然后用`pd.read_clipboard()`从剪贴板读取数据。

In [10]:
students = pd.read_clipboard(sep='\t')
students

,姓名,Task 0,Task 1
0,Lucifer,九阴真经上卷,九阴真经下卷
1,Tyelcie,九阴真经下卷,真经后面的小诗


如果要群发消息，注意检查任务序列和消息模板：

In [11]:
task_seq = 'Task 1'
msg_template = 'Hello，{}同学，接下来三天你需要完成的任务是：\n\n{}'
sc.send_task_group(task_seq, msg_template, students)

如果需要单独发消息，注意修改学生姓名、任务序列和消息模板：

In [13]:
student = 'Tyelcie'
task_seq = 'Task 1'
msg_template = 'Hello，{}同学，接下来三天你需要完成的任务是：\n\n{}'
sc.send_task_personal(student, task_seq, msg_template, students)

暂时就这样。